In [1]:
import os 

import pandas as pd
import numpy as np

import bbg_symbology as bbs
from bbg_loader_core import *

In [2]:
def merge_dict(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

class ExpiredAliasService:
    def __init__(self, src=None, dbfldr=None):
        
        self._src = src
        self._dbfldr = dbfldr
        
        if src is not None:
            self.expired = list(set(pd.read_csv(src, header=None)[1]))
            
            if self._dbfldr is not None:
                dblist = self.get_db_list()
                self.expired = [e for e in self.expired if e in dblist]
        else:
            self.expired = []
    
    def get_db_list(self):
        extension = '.pickle'
        return [f.replace(extension, '') for f in os.listdir(self._dbfldr) if f.endswith(extension)]
    
    def __contains__(self, key):
        return key in self.expired
    
    def __repr__(self):
        return '<{}: {}>'.format(self.__class__, str(self.expired))
    
    def add_expired(self, aliases):
        if isinstance(aliases, list):
            self.expired = self.expired + aliases
        else:
            self.expired.extend([aliases])
        
    def save(self, fname=None):
        if fname is None:
            fname = self._src
            
        if fname is None:
            raise Error('No Filename Specified!')
            
        pd.Series(self.expired).to_csv(fname)
    

In [ ]:
def generate_futures_list(out_path):
    
    # init - flds & default params
    ts_flds =   ['px_last', 'open_int', 'volume']
    meta_flds = ['LAST_TRADEABLE_DT']

    default_params = {'local_path':  out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}
    
    # get all unique roots (alias roots)
    roots = list(FutAliasRef._fut_ref.Alias)
    
    # store outputs
    fut_list = {}
    
    # loop through roots
    for root in roots:
        
        # get all bbg, alias tckrs
        bbg_tckrs  = FutChainRef.get_futures_chain(root)
        alias_tckrs = [FutAliasRef.bbg_to_alias_tckr(tckr) for tckr in bbg_tckrs]
        tckr_list = zip(bbg_tckrs, alias_tckrs)
        
        # loop through ticker combos... and return object
        for bbg, alias in tckr_list:
            if alias not in ExpiredFutures:
                p = merge_dict({'bb_tckr': bbg, 'alias': alias}, default_params)
                fut_list[alias] = BbgSecurity(**p)
                
    return fut_list


def generate_bbg_list(src_file, out_path, ts_flds=['px_last'], meta_flds=['NAME','LONG_COMP_NAME']):
    tckrList = pd.read_csv(src_file)
    default_params = {'local_path': out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}

    secList = {}    
    for (i, (bbg,alias)) in tckrList.iterrows():
        d = merge_dict({'bb_tckr': bbg, 'alias': alias}, default_params)
        secList[bbg] = BbgSecurity(**d)

    return secList


def generate_generic_fut_list(out_path, ts_flds=['px_last'], meta_flds=['NAME','LONG_COMP_NAME']):

    def gen_obj(broot, rmtd, yk, ng, alias_root, alias_suff, dparams):
        bbg_tckr   = broot.upper()+ str(ng) + ' ' + rmtd + ' ' + yk
        alias_tckr = alias_root + str(ng) + '.' + alias_suff

        d = merge_dict({'bb_tckr': bbg_tckr, 'alias': alias_tckr}, dparams)
        return alias_tckr, BbgSecurity(**d)
    
    ts_flds =   ['px_last', 'open_int', 'volume']
    meta_flds = ['NAME', 'PX_POS_MULT_FACTOR', 'EXCH_CODE', 'FUT_EXCH_NAME_LONG', 
                 'CRNCY', 'FUT_TICK_SIZE', 'FUT_GEN_MONTH']

    default_params_ridx = {'local_path': out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}
    default_params_px   = {'local_path': out_path, 'ts_flds': ts_flds[:1], 'meta_flds': meta_flds[:1]}
    
    
    n = 1
    secList = {}    
    for idx, (root, num_gen, yk, alias) in FutAliasRef._fut_ref.iterrows():
        # Return Index
        atckr, bbobj = gen_obj(root, 'B:00_0_D', yk, 1, alias, 'bbd', default_params_ridx)
        secList[atckr] = bbobj
        
        # Price 
        atckr, bbobj = gen_obj(root, 'B:00_0_N', yk, 1, alias, 'bbd.px', default_params_px)
        secList[atckr] = bbobj
        
        # Return Index
        atckr, bbobj = gen_obj(root, 'B:00_0_D', yk, num_gen, alias, 'bbd', default_params_px)
        secList[atckr] = bbobj
        
        # Price
        atckr, bbobj = gen_obj(root, 'B:00_0_N', yk, num_gen, alias, 'bbd.px', default_params_px)
        secList[atckr] = bbobj

    return secList

def update_bbg_securities(bbglist):
    expired = []
    error_list = []
    print('--------------UPDATING {}----------------'.format(db))
    for i, (alias, bbgObj) in enumerate(bbglist.items()):
        print('\n(' + str(i) + ') ' + str(bbgObj))

        try:
            bbgObj.update()
            if bbgObj.is_expired:
                expired.append(bbgObj.alias)
        except:
            try: 
                print('...Error with {}, attempting to load from scratch'.format(bbgObj.bb_tckr))
                bbgObj.load_from_scratch()
                bbgObj.save()
                print('...success!')

                if bbgObj.is_expired:
                    expired.append(bbgObj.alias)
            except:
                print('FATAL ERROR WITH {}'.format(bbgObj.bb_tckr))
                error_list.extend(bbgObj)
    
    return expired, error_list

In [3]:
def alias_to_date(alias):
    dtinfo = alias.split('.')[1]
    month_str = dtinfo[0]
    yr_str = dtinfo[1:]
    
    MONTH_REF = 'FGHJKMNQUVXZ'
    month =  MONTH_REF.find(month_str) + 1
    
    yr =  int(yr_str)
    if yr > 50:
        yr = yr + 1900
    else:
        yr = yr + 2000

    return pd.datetime(yr, month, 1)

def get_sec_health(bbg):
    #bbg.load_local_data()
    health = bbg.ts.count()
    
    if 'LAST_TRADEABLE_DT' in bbg.meta:
        health['exp_error'] = np.abs((bbg.meta.LAST_TRADEABLE_DT - alias_to_date(bbg.alias)).days)
    
    try: 
        health['first_dt'] = (bbg.ts.first_valid_index()).strftime('%Y-%m-%d')
    except: 
        health['first_dt'] = None
        
    try: 
        health['last_dt'] = (bbg.ts.last_valid_index()).strftime('%Y-%m-%d')
    except:
        health['last_dt'] = None
        
    health.name = bbg.alias
    return health

def filter_health(df):

    err = pd.DataFrame()
    
    ts_flds = ['px_last', 'open_int', 'volume']

    for fld in ts_flds:
        if fld in df:
            err[fld] = df.px_last==0

    if 'exp_error' in df:
        err['exp'] = df.exp_error > 50

    return df[err.any(axis=1)]

# TODO THIS FUNCTION NEEDS WORK
def export_db_health(dbname, dbfldr='../_bbgDB/'):
        DIR = dbfldr + dbname + '/'

        flist = [f for f in os.listdir(DIR) if f.endswith('.pickle')]
        health_list = []
        
        for f in flist:
            print(f)
            bbg = BbgSecurity.from_file(DIR+f)
            health_list.append(get_sec_health(bbg))

        healthdf.update({dbname:pd.DataFrame(health_list)})
        filter_health(healthdf).to_excel('../_bbgDB/health_' + dbname + '.xlsx')

In [4]:
INPUT_PATH = '../_in/'

FUTURES_MONTHS = {m:i+1 for i,m in enumerate('FGHJKMNQUVXZ')}

FutChainRef    = bbs.FuturesChainReference(INPUT_PATH + 'futures_historical_chain.csv')
FutAliasRef    = bbs.FuturesAliasService(INPUT_PATH + 'fut_roots.csv')
BbgTckrService = bbs.BloombergTckrService()
ExpiredFutures = ExpiredAliasService('../_in/expired_aliases.csv', '../_bbgDB/Futures/')


In [5]:
BBG_UPDATE_LIST = {'CoT':           generate_bbg_list(src_file=INPUT_PATH + 'CoT.csv',           out_path='../_bbgDB/CoT/'),
                   'Index':         generate_bbg_list(src_file=INPUT_PATH + 'Index.csv',         out_path='../_bbgDB/Index/'),
                   'FX':            generate_bbg_list(src_file=INPUT_PATH + 'FX.csv',            out_path='../_bbgDB/FX/'), 
                   'InterestRates': generate_bbg_list(src_file=INPUT_PATH + 'InterestRates.csv', out_path='../_bbgDB/InterestRates/'),
                   'Futures_gen':   generate_generic_fut_list(                                   out_path='../_bbgDB/Futures_gen/'),
                   'Futures':       generate_futures_list(                                       out_path='../_bbgDB/Futures/')}

# print # securities in each
for name, db in BBG_UPDATE_LIST.items():
    print(name, len(db))


('Index', 4)
('CoT', 14)
('FX', 68)
('Futures', 2094)
('Futures_gen', 230)
('InterestRates', 46)


In [26]:
tckrs = list(BBG_UPDATE_LIST['Futures'].keys())
tckrs_ = [tckr for tckr in tckrs if tckr.split('.')[0]=='ng'

In [7]:
tckr = 'gc.N84'
bb = BbgSecurity.from_file('../_bbgDB/Futures/' + tckr + '.pickle')
bb.is_expired

True

In [8]:
expired = {}
error_list = {}

for db, bbglist in BBG_UPDATE_LIST.items():
    expired[db], error_list[db] = update_bbg_securities(bbglist)

--------------UPDATING Index----------------

(0) <BBG SECURITY: tckr=SPGSCIP Index, alias=gsci.er, local_file=../_bbgDB/Index/>
...updating security SPGSCIP Index
...loading local file <../_bbgDB/Index/gsci.er.pickle> for security=SPGSCIP Index
...updating timeseries for SPGSCIP Index --> loading data from 09/19/2019 to 09/26/2019
...saving SPGSCIP Index to local file <../_bbgDB/Index/gsci.er.pickle>

(1) <BBG SECURITY: tckr=SPGSCITR Index, alias=gsci.tr, local_file=../_bbgDB/Index/>
...updating security SPGSCITR Index
...loading local file <../_bbgDB/Index/gsci.tr.pickle> for security=SPGSCITR Index
...updating timeseries for SPGSCITR Index --> loading data from 09/19/2019 to 09/26/2019
...saving SPGSCITR Index to local file <../_bbgDB/Index/gsci.tr.pickle>

(2) <BBG SECURITY: tckr=NEIXSTTI Index, alias=sgstti, local_file=../_bbgDB/Index/>
...updating security NEIXSTTI Index
...loading local file <../_bbgDB/Index/sgstti.pickle> for security=NEIXSTTI Index
...updating timeseries for N

..security SEKUSDCR Index is up to date! Done!

(12) <BBG SECURITY: tckr=BPPPCPJP Index, alias=jpy.cppp, local_file=../_bbgDB/FX/>
...updating security BPPPCPJP Index
...loading local file <../_bbgDB/FX/jpy.cppp.pickle> for security=BPPPCPJP Index
...updating timeseries for BPPPCPJP Index --> loading data from 04/30/2019 to 09/26/2019
...saving BPPPCPJP Index to local file <../_bbgDB/FX/jpy.cppp.pickle>

(13) <BBG SECURITY: tckr=AUD1M Curncy, alias=aud.1m, local_file=../_bbgDB/FX/>
...updating security AUD1M Curncy
...loading local file <../_bbgDB/FX/aud.1m.pickle> for security=AUD1M Curncy
..security AUD1M Curncy is up to date! Done!

(14) <BBG SECURITY: tckr=BPPPPPAU Index, alias=aud.pppp, local_file=../_bbgDB/FX/>
...updating security BPPPPPAU Index
...loading local file <../_bbgDB/FX/aud.pppp.pickle> for security=BPPPPPAU Index
...updating timeseries for BPPPPPAU Index --> loading data from 02/28/2019 to 09/26/2019
...saving BPPPPPAU Index to local file <../_bbgDB/FX/aud.pppp.pickl

...saving BPPPCPSE Index to local file <../_bbgDB/FX/sek.cppp.pickle>

(50) <BBG SECURITY: tckr=CHF3M Curncy, alias=chf.3m, local_file=../_bbgDB/FX/>
...updating security CHF3M Curncy
...loading local file <../_bbgDB/FX/chf.3m.pickle> for security=CHF3M Curncy
..security CHF3M Curncy is up to date! Done!

(51) <BBG SECURITY: tckr=NOK3M Curncy, alias=nok.3m, local_file=../_bbgDB/FX/>
...updating security NOK3M Curncy
...loading local file <../_bbgDB/FX/nok.3m.pickle> for security=NOK3M Curncy
..security NOK3M Curncy is up to date! Done!

(52) <BBG SECURITY: tckr=AUD Curncy, alias=aud.spot, local_file=../_bbgDB/FX/>
...updating security AUD Curncy
...loading local file <../_bbgDB/FX/aud.spot.pickle> for security=AUD Curncy
..security AUD Curncy is up to date! Done!

(53) <BBG SECURITY: tckr=NOK1M Curncy, alias=nok.1m, local_file=../_bbgDB/FX/>
...updating security NOK1M Curncy
...loading local file <../_bbgDB/FX/nok.1m.pickle> for security=NOK1M Curncy
..security NOK1M Curncy is up to da

C:\Anaconda\envs\python2\lib\site-packages\tia\bbg\v3api.py:409: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  frame = frame.reindex_axis(self.request.fields, axis=1)


...saving RUK99 Curncy to local file <../_bbgDB/Futures/ru.K99.pickle>

(1) <BBG SECURITY: tckr=QSH2 Comdty, alias=qs.H22, local_file=../_bbgDB/Futures/>
...updating security QSH2 Comdty
...loading local file <../_bbgDB/Futures/qs.H22.pickle> for security=QSH2 Comdty
..security QSH2 Comdty is up to date! Done!

(2) <BBG SECURITY: tckr=QSH3 Comdty, alias=qs.H23, local_file=../_bbgDB/Futures/>
...updating security QSH3 Comdty
...loading local file <../_bbgDB/Futures/qs.H23.pickle> for security=QSH3 Comdty
..security QSH3 Comdty is up to date! Done!

(3) <BBG SECURITY: tckr=QSH0 Comdty, alias=qs.H20, local_file=../_bbgDB/Futures/>
...updating security QSH0 Comdty
...loading local file <../_bbgDB/Futures/qs.H20.pickle> for security=QSH0 Comdty
..security QSH0 Comdty is up to date! Done!

(4) <BBG SECURITY: tckr=QSH1 Comdty, alias=qs.H21, local_file=../_bbgDB/Futures/>
...updating security QSH1 Comdty
...loading local file <../_bbgDB/Futures/qs.H21.pickle> for security=QSH1 Comdty
..securit

...

...updating timeseries for LA1 B:00_0_D Comdty --> loading data from 09/19/2019 to 09/26/2019
...saving LA1 B:00_0_D Comdty to local file <../_bbgDB/Futures_gen/la1.bbd.pickle>

(149) <BBG SECURITY: tckr=EC1 B:00_0_D Curncy, alias=ec1.bbd, local_file=../_bbgDB/Futures_gen/>
...updating security EC1 B:00_0_D Curncy
...loading local file <../_bbgDB/Futures_gen/ec1.bbd.pickle> for security=EC1 B:00_0_D Curncy
..security EC1 B:00_0_D Curncy is up to date! Done!

(150) <BBG SECURITY: tckr=MW6 B:00_0_N Comdty, alias=mw6.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security MW6 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/mw6.bbd.px.pickle> for security=MW6 B:00_0_N Comdty
..security MW6 B:00_0_N Comdty is up to date! Done!

(151) <BBG SECURITY: tckr=JB1 B:00_0_N Comdty, alias=jb1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security JB1 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/jb1.bbd.px.pickle> for security=JB1 B:00_0_N Comdty
..securi

...saving LA1 B:00_0_N Comdty to local file <../_bbgDB/Futures_gen/la1.bbd.px.pickle>

(178) <BBG SECURITY: tckr=CN1 B:00_0_N Comdty, alias=cn1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security CN1 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/cn1.bbd.px.pickle> for security=CN1 B:00_0_N Comdty
..security CN1 B:00_0_N Comdty is up to date! Done!

(179) <BBG SECURITY: tckr=RX1 B:00_0_N Comdty, alias=rx1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security RX1 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/rx1.bbd.px.pickle> for security=RX1 B:00_0_N Comdty
..security RX1 B:00_0_N Comdty is up to date! Done!

(180) <BBG SECURITY: tckr=CT1 B:00_0_N Comdty, alias=ct1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security CT1 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/ct1.bbd.px.pickle> for security=CT1 B:00_0_N Comdty
...updating timeseries for CT1 B:00_0_N Comdty --> loading data from 09/19/2019 to 09/26/2019

..security Z 1 B:00_0_D Index is up to date! Done!

(217) <BBG SECURITY: tckr=SM1 B:00_0_D Comdty, alias=sym1.bbd, local_file=../_bbgDB/Futures_gen/>
...updating security SM1 B:00_0_D Comdty
...loading local file <../_bbgDB/Futures_gen/sym1.bbd.pickle> for security=SM1 B:00_0_D Comdty
..security SM1 B:00_0_D Comdty is up to date! Done!

(218) <BBG SECURITY: tckr=CT6 B:00_0_N Comdty, alias=ct6.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security CT6 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/ct6.bbd.px.pickle> for security=CT6 B:00_0_N Comdty
...updating timeseries for CT6 B:00_0_N Comdty --> loading data from 09/19/2019 to 09/26/2019
...saving CT6 B:00_0_N Comdty to local file <../_bbgDB/Futures_gen/ct6.bbd.px.pickle>

(219) <BBG SECURITY: tckr=EC1 B:00_0_N Curncy, alias=ec1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security EC1 B:00_0_N Curncy
...loading local file <../_bbgDB/Futures_gen/ec1.bbd.px.pickle> for security=EC1 B:00_0_N Curncy
..se

...saving EURR002W Index to local file <../_bbgDB/InterestRates/eu.cb.pickle>

(17) <BBG SECURITY: tckr=GCAN7YR Index Index, alias=cn.7y, local_file=../_bbgDB/InterestRates/>
...updating security GCAN7YR Index Index
...loading local file <../_bbgDB/InterestRates/cn.7y.pickle> for security=GCAN7YR Index Index
..security GCAN7YR Index Index is up to date! Done!

(18) <BBG SECURITY: tckr=EUR001M Index, alias=eu.1m, local_file=../_bbgDB/InterestRates/>
...updating security EUR001M Index
...loading local file <../_bbgDB/InterestRates/eu.1m.pickle> for security=EUR001M Index
...updating timeseries for EUR001M Index --> loading data from 09/19/2019 to 09/26/2019
...saving EUR001M Index to local file <../_bbgDB/InterestRates/eu.1m.pickle>

(19) <BBG SECURITY: tckr=GUKG2 Index Index, alias=uk.2y, local_file=../_bbgDB/InterestRates/>
...updating security GUKG2 Index Index
...loading local file <../_bbgDB/InterestRates/uk.2y.pickle> for security=GUKG2 Index Index
..security GUKG2 Index Index is u

...saving SZLTHIGH Index to local file <../_bbgDB/InterestRates/ch.cb.pickle>

(45) <BBG SECURITY: tckr=GNZGB2 Index, alias=nz.2y, local_file=../_bbgDB/InterestRates/>
...updating security GNZGB2 Index
...loading local file <../_bbgDB/InterestRates/nz.2y.pickle> for security=GNZGB2 Index
..security GNZGB2 Index is up to date! Done!


In [27]:
ExpiredFutures.add_expired(expired['Futures'])
ExpiredFutures.save()       

In [28]:
databases = ['Futures', 'Futures_gen', 'FX', 'CoT', 'Index', 'InterestRates' ]
health = {}

for db in databases:
    health[db] = export_db_health(db)


ad.F10.pickle
ad.F18.pickle
ad.F19.pickle
ad.F20.pickle
ad.G18.pickle
ad.G19.pickle
ad.H00.pickle
ad.H01.pickle
ad.H02.pickle
ad.H03.pickle
ad.H04.pickle
ad.H05.pickle
ad.H06.pickle
ad.H07.pickle
ad.H08.pickle
ad.H09.pickle
ad.H10.pickle
ad.H11.pickle
ad.H12.pickle
ad.H13.pickle
ad.H14.pickle
ad.H15.pickle
ad.H16.pickle
ad.H17.pickle
ad.H18.pickle
ad.H19.pickle
ad.H20.pickle
ad.H21.pickle
ad.H22.pickle
ad.H23.pickle
ad.H24.pickle
ad.H87.pickle
ad.H88.pickle
ad.H89.pickle
ad.H90.pickle
ad.H91.pickle
ad.H92.pickle
ad.H93.pickle
ad.H94.pickle
ad.H95.pickle
ad.H96.pickle
ad.H97.pickle
ad.H98.pickle
ad.H99.pickle
ad.J17.pickle
ad.J18.pickle
ad.J19.pickle
ad.K17.pickle
ad.K18.pickle
ad.K19.pickle
ad.M00.pickle
ad.M01.pickle
ad.M02.pickle
ad.M03.pickle
ad.M04.pickle
ad.M05.pickle
ad.M06.pickle
ad.M07.pickle
ad.M08.pickle
ad.M09.pickle
ad.M10.pickle
ad.M11.pickle
ad.M12.pickle
ad.M13.pickle
ad.M14.pickle
ad.M15.pickle
ad.M16.pickle
ad.M17.pickle
ad.M18.pickle
ad.M19.pickle
ad.M20.pickle
ad.M21

bo.H85.pickle
bo.H86.pickle
bo.H87.pickle
bo.H88.pickle
bo.H89.pickle
bo.H90.pickle
bo.H91.pickle
bo.H92.pickle
bo.H93.pickle
bo.H94.pickle
bo.H95.pickle
bo.H96.pickle
bo.H97.pickle
bo.H98.pickle
bo.H99.pickle
bo.K00.pickle
bo.K01.pickle
bo.K02.pickle
bo.K03.pickle
bo.K04.pickle
bo.K05.pickle
bo.K06.pickle
bo.K07.pickle
bo.K08.pickle
bo.K09.pickle
bo.K10.pickle
bo.K11.pickle
bo.K12.pickle
bo.K13.pickle
bo.K14.pickle
bo.K15.pickle
bo.K16.pickle
bo.K17.pickle
bo.K18.pickle
bo.K19.pickle
bo.K20.pickle
bo.K21.pickle
bo.K60.pickle
bo.K61.pickle
bo.K62.pickle
bo.K63.pickle
bo.K64.pickle
bo.K65.pickle
bo.K66.pickle
bo.K67.pickle
bo.K68.pickle
bo.K69.pickle
bo.K70.pickle
bo.K71.pickle
bo.K72.pickle
bo.K73.pickle
bo.K74.pickle
bo.K75.pickle
bo.K76.pickle
bo.K77.pickle
bo.K78.pickle
bo.K79.pickle
bo.K80.pickle
bo.K81.pickle
bo.K82.pickle
bo.K83.pickle
bo.K84.pickle
bo.K85.pickle
bo.K86.pickle
bo.K87.pickle
bo.K88.pickle
bo.K89.pickle
bo.K90.pickle
bo.K91.pickle
bo.K92.pickle
bo.K93.pickle
bo.K94

c.H67.pickle
c.H68.pickle
c.H69.pickle
c.H70.pickle
c.H71.pickle
c.H72.pickle
c.H73.pickle
c.H74.pickle
c.H75.pickle
c.H76.pickle
c.H77.pickle
c.H78.pickle
c.H79.pickle
c.H80.pickle
c.H81.pickle
c.H82.pickle
c.H83.pickle
c.H84.pickle
c.H85.pickle
c.H86.pickle
c.H87.pickle
c.H88.pickle
c.H89.pickle
c.H90.pickle
c.H91.pickle
c.H92.pickle
c.H93.pickle
c.H94.pickle
c.H95.pickle
c.H96.pickle
c.H97.pickle
c.H98.pickle
c.H99.pickle
c.K00.pickle
c.K01.pickle
c.K02.pickle
c.K03.pickle
c.K04.pickle
c.K05.pickle
c.K06.pickle
c.K07.pickle
c.K08.pickle
c.K09.pickle
c.K10.pickle
c.K11.pickle
c.K12.pickle
c.K13.pickle
c.K14.pickle
c.K15.pickle
c.K16.pickle
c.K17.pickle
c.K18.pickle
c.K19.pickle
c.K20.pickle
c.K21.pickle
c.K60.pickle
c.K61.pickle
c.K62.pickle
c.K63.pickle
c.K64.pickle
c.K65.pickle
c.K66.pickle
c.K67.pickle
c.K68.pickle
c.K69.pickle
c.K70.pickle
c.K71.pickle
c.K72.pickle
c.K73.pickle
c.K74.pickle
c.K75.pickle
c.K76.pickle
c.K77.pickle
c.K78.pickle
c.K79.pickle
c.K80.pickle
c.K81.pickle

cd.H19.pickle
cd.H20.pickle
cd.H21.pickle
cd.H22.pickle
cd.H23.pickle
cd.H24.pickle
cd.H87.pickle
cd.H88.pickle
cd.H89.pickle
cd.H90.pickle
cd.H91.pickle
cd.H92.pickle
cd.H93.pickle
cd.H94.pickle
cd.H95.pickle
cd.H96.pickle
cd.H97.pickle
cd.H98.pickle
cd.H99.pickle
cd.J17.pickle
cd.J18.pickle
cd.J19.pickle
cd.K17.pickle
cd.K18.pickle
cd.K19.pickle
cd.M00.pickle
cd.M01.pickle
cd.M02.pickle
cd.M03.pickle
cd.M04.pickle
cd.M05.pickle
cd.M06.pickle
cd.M07.pickle
cd.M08.pickle
cd.M09.pickle
cd.M10.pickle
cd.M11.pickle
cd.M12.pickle
cd.M13.pickle
cd.M14.pickle
cd.M15.pickle
cd.M16.pickle
cd.M17.pickle
cd.M18.pickle
cd.M19.pickle
cd.M20.pickle
cd.M21.pickle
cd.M22.pickle
cd.M23.pickle
cd.M24.pickle
cd.M87.pickle
cd.M88.pickle
cd.M89.pickle
cd.M90.pickle
cd.M91.pickle
cd.M92.pickle
cd.M93.pickle
cd.M94.pickle
cd.M95.pickle
cd.M96.pickle
cd.M97.pickle
cd.M98.pickle
cd.M99.pickle
cd.N17.pickle
cd.N18.pickle
cd.N19.pickle
cd.Q17.pickle
cd.Q18.pickle
cd.Q19.pickle
cd.U00.pickle
cd.U01.pickle
cd.U02

cl.G11.pickle
cl.G12.pickle
cl.G13.pickle
cl.G14.pickle
cl.G15.pickle
cl.G16.pickle
cl.G17.pickle
cl.G18.pickle
cl.G19.pickle
cl.G20.pickle
cl.G21.pickle
cl.G22.pickle
cl.G23.pickle
cl.G24.pickle
cl.G25.pickle
cl.G26.pickle
cl.G27.pickle
cl.G28.pickle
cl.G29.pickle
cl.G30.pickle
cl.G84.pickle
cl.G85.pickle
cl.G86.pickle
cl.G87.pickle
cl.G88.pickle
cl.G89.pickle
cl.G90.pickle
cl.G91.pickle
cl.G92.pickle
cl.G93.pickle
cl.G94.pickle
cl.G95.pickle
cl.G96.pickle
cl.G97.pickle
cl.G98.pickle
cl.G99.pickle
cl.H00.pickle
cl.H01.pickle
cl.H02.pickle
cl.H03.pickle
cl.H04.pickle
cl.H05.pickle
cl.H06.pickle
cl.H07.pickle
cl.H08.pickle
cl.H09.pickle
cl.H10.pickle
cl.H11.pickle
cl.H12.pickle
cl.H13.pickle
cl.H14.pickle
cl.H15.pickle
cl.H16.pickle
cl.H17.pickle
cl.H18.pickle
cl.H19.pickle
cl.H20.pickle
cl.H21.pickle
cl.H22.pickle
cl.H23.pickle
cl.H24.pickle
cl.H25.pickle
cl.H26.pickle
cl.H27.pickle
cl.H28.pickle
cl.H29.pickle
cl.H84.pickle
cl.H85.pickle
cl.H86.pickle
cl.H87.pickle
cl.H88.pickle
cl.H89

...

sym.U66.pickle
sym.U67.pickle
sym.U68.pickle
sym.U69.pickle
sym.U70.pickle
sym.U71.pickle
sym.U72.pickle
sym.U73.pickle
sym.U74.pickle
sym.U75.pickle
sym.U76.pickle
sym.U77.pickle
sym.U78.pickle
sym.U79.pickle
sym.U80.pickle
sym.U81.pickle
sym.U82.pickle
sym.U83.pickle
sym.U84.pickle
sym.U85.pickle
sym.U86.pickle
sym.U87.pickle
sym.U88.pickle
sym.U89.pickle
sym.U90.pickle
sym.U91.pickle
sym.U92.pickle
sym.U93.pickle
sym.U94.pickle
sym.U95.pickle
sym.U96.pickle
sym.U97.pickle
sym.U98.pickle
sym.U99.pickle
sym.V00.pickle
sym.V01.pickle
sym.V02.pickle
sym.V03.pickle
sym.V04.pickle
sym.V05.pickle
sym.V06.pickle
sym.V07.pickle
sym.V08.pickle
sym.V09.pickle
sym.V10.pickle
sym.V11.pickle
sym.V12.pickle
sym.V13.pickle
sym.V14.pickle
sym.V15.pickle
sym.V16.pickle
sym.V17.pickle
sym.V18.pickle
sym.V19.pickle
sym.V20.pickle
sym.V21.pickle
sym.V22.pickle
sym.V60.pickle
sym.V61.pickle
sym.V62.pickle
sym.V63.pickle
sym.V64.pickle
sym.V65.pickle
sym.V66.pickle
sym.V67.pickle
sym.V68.pickle
sym.V69.pi

ux.G19.pickle
ux.G20.pickle
ux.H05.pickle
ux.H06.pickle
ux.H07.pickle
ux.H08.pickle
ux.H09.pickle
ux.H10.pickle
ux.H11.pickle
ux.H12.pickle
ux.H13.pickle
ux.H14.pickle
ux.H15.pickle
ux.H16.pickle
ux.H17.pickle
ux.H18.pickle
ux.H19.pickle
ux.H20.pickle
ux.J06.pickle
ux.J07.pickle
ux.J08.pickle
ux.J09.pickle
ux.J10.pickle
ux.J11.pickle
ux.J12.pickle
ux.J13.pickle
ux.J14.pickle
ux.J15.pickle
ux.J16.pickle
ux.J17.pickle
ux.J18.pickle
ux.J19.pickle
ux.J20.pickle
ux.K04.pickle
ux.K05.pickle
ux.K06.pickle
ux.K07.pickle
ux.K08.pickle
ux.K09.pickle
ux.K10.pickle
ux.K11.pickle
ux.K12.pickle
ux.K13.pickle
ux.K14.pickle
ux.K15.pickle
ux.K16.pickle
ux.K17.pickle
ux.K18.pickle
ux.K19.pickle
ux.K20.pickle
ux.M04.pickle
ux.M05.pickle
ux.M06.pickle
ux.M07.pickle
ux.M08.pickle
ux.M09.pickle
ux.M10.pickle
ux.M11.pickle
ux.M12.pickle
ux.M13.pickle
ux.M14.pickle
ux.M15.pickle
ux.M16.pickle
ux.M17.pickle
ux.M18.pickle
ux.M19.pickle
ux.M20.pickle
ux.N04.pickle
ux.N06.pickle
ux.N07.pickle
ux.N08.pickle
ux.N09

xbw.G00.pickle
xbw.G01.pickle
xbw.G02.pickle
xbw.G03.pickle
xbw.G04.pickle
xbw.G05.pickle
xbw.G06.pickle
xbw.G07.pickle
xbw.G08.pickle
xbw.G09.pickle
xbw.G10.pickle
xbw.G11.pickle
xbw.G12.pickle
xbw.G13.pickle
xbw.G14.pickle
xbw.G15.pickle
xbw.G16.pickle
xbw.G17.pickle
xbw.G18.pickle
xbw.G19.pickle
xbw.G20.pickle
xbw.G21.pickle
xbw.G22.pickle
xbw.G87.pickle
xbw.G88.pickle
xbw.G89.pickle
xbw.G90.pickle
xbw.G91.pickle
xbw.G92.pickle
xbw.G93.pickle
xbw.G94.pickle
xbw.G95.pickle
xbw.G96.pickle
xbw.G97.pickle
xbw.G98.pickle
xbw.G99.pickle
xbw.H00.pickle
xbw.H01.pickle
xbw.H02.pickle
xbw.H03.pickle
xbw.H04.pickle
xbw.H05.pickle
xbw.H06.pickle
xbw.H07.pickle
xbw.H08.pickle
xbw.H09.pickle
xbw.H10.pickle
xbw.H11.pickle
xbw.H12.pickle
xbw.H13.pickle
xbw.H14.pickle
xbw.H15.pickle
xbw.H16.pickle
xbw.H17.pickle
xbw.H18.pickle
xbw.H19.pickle
xbw.H20.pickle
xbw.H21.pickle
xbw.H22.pickle
xbw.H87.pickle
xbw.H88.pickle
xbw.H89.pickle
xbw.H90.pickle
xbw.H91.pickle
xbw.H92.pickle
xbw.H93.pickle
xbw.H94.pi

xp.H14.pickle
xp.H15.pickle
xp.H16.pickle
xp.H17.pickle
xp.H18.pickle
xp.H19.pickle
xp.H20.pickle
xp.H21.pickle
xp.J07.pickle
xp.J11.pickle
xp.J12.pickle
xp.J13.pickle
xp.J14.pickle
xp.J15.pickle
xp.J16.pickle
xp.J17.pickle
xp.J18.pickle
xp.J19.pickle
xp.K07.pickle
xp.K11.pickle
xp.K12.pickle
xp.K13.pickle
xp.K14.pickle
xp.K15.pickle
xp.K16.pickle
xp.K17.pickle
xp.K18.pickle
xp.K19.pickle
xp.M00.pickle
xp.M01.pickle
xp.M02.pickle
xp.M03.pickle
xp.M04.pickle
xp.M05.pickle
xp.M06.pickle
xp.M07.pickle
xp.M08.pickle
xp.M09.pickle
xp.M10.pickle
xp.M11.pickle
xp.M12.pickle
xp.M13.pickle
xp.M14.pickle
xp.M15.pickle
xp.M16.pickle
xp.M17.pickle
xp.M18.pickle
xp.M19.pickle
xp.M20.pickle
xp.N07.pickle
xp.N10.pickle
xp.N11.pickle
xp.N12.pickle
xp.N13.pickle
xp.N14.pickle
xp.N15.pickle
xp.N16.pickle
xp.N17.pickle
xp.N18.pickle
xp.N19.pickle
xp.Q07.pickle
xp.Q10.pickle
xp.Q11.pickle
xp.Q12.pickle
xp.Q13.pickle
xp.Q14.pickle
xp.Q15.pickle
xp.Q16.pickle
xp.Q17.pickle
xp.Q18.pickle
xp.Q19.pickle
xp.U00

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [39]:
import os
import zipfile



'../_out/bbgDB-2019-09-27.zip'

In [42]:
zipfldr = '../_out/'
zipfilename = 'bbgDB-' + datetime.now().strftime('%Y-%m-%d') + '.zip'


zf = zipfile.ZipFile(zipfldr + zipfilename, 'w', zipfile.ZIP_DEFLATED)
for dirname, subdirs, files in os.walk("../_bbgDB"):    
    zf.write(dirname)
    for filename in files:
        zf.write(os.path.join(dirname, filename))
zf.close()
